In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [40]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval
import tensorflow_quantum as tfq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
translator = CirqTranslater(3, untouchable_blocks = [1], discard_qubits=[2])
simplifier = Simplifier(translator)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.untouchable_blocks, untouchable_qubits = [2])



etas = [1.,0.01]

args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)

minimizer = Minimizer(translator, mode="discrimination", params=etas)
killer = GateKiller(translator, mode="discrimination", params=etas)

channel_db = amplitude_damping_db(translator, qubits_ind=[0,inserter.untouchable_qubits[0]], eta=1, block_id = translator.untouchable_blocks[0])
circuit_db = concatenate_dbs([u2_db(translator,0,1, block_id=0), channel_db, u2_db(translator,0,1, block_id=2)])


In [87]:
circuit, circuit_db = translator.give_circuit(circuit_db)

In [88]:
circuit, circuit_db = translator.give_circuit(circuit_db)
minimized_db, [cost, resolver, history_training] = minimizer.variational(circuit_db)
evaluator.add_step(minimized_db, cost, relevant=True, operation="variational", history = history_training.history["cost"])

In [89]:
simplified_db, ns =  simplifier.reduce_circuit(minimized_db)
newcost = minimizer.give_cost(simplified_db)

In [90]:
evaluator.add_step(simplified_db, newcost, relevant=False, operation="simplification", history = ns)

In [91]:
killed_db, killed_cost, murders = killer.remove_irrelevant_gates(cost,simplified_db)

kill 1qbit gate, try 0/28. Increased by: -3.2657194424245972e-06%
kill 1qbit gate, try 1/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 2/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 3/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 4/28. Increased by: -2.0410745946719544e-06%
kill 1qbit gate, try 5/28. Increased by: -2.0410745946719544e-06%
kill 1qbit gate, try 6/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 7/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 8/28. Increased by: -8.164298606061493e-07%
kill 1qbit gate, try 9/28. Increased by: -1.6328597212122986e-06%
kill 1qbit gate, try 10/28. Increased by: -2.44928946813161e-06%
kill 1qbit gate, try 11/28. Increased by: -0.006440815050154924%
kill 1qbit gate, try 12/28. Increased by: 0.0%


In [92]:
evaluator.add_step(killed_db, killed_cost, relevant=False, operation="gate_removals", history = murders)

In [93]:
circuit_db = killed_db.copy()

In [95]:
mutated_db, number_mutations = inserter.mutate(circuit_db)
mutated_cost = minimizer.give_cost(mutated_db)
evaluator.add_step(mutated_db, mutated_cost, relevant=False, operation="mutation", history = number_mutations)

simplified_db, ns =  simplifier.reduce_circuit(mutated_db)
simplified_cost = minimizer.give_cost(simplified_db)
evaluator.add_step(simplified_db, simplified_cost, relevant=False, operation="simplification", history = ns)

minimized_db, [cost, resolver, history_training] = minimizer.variational(simplified_db)
evaluator.add_step(simplified_db, simplified_cost, relevant=False, operation="variational", history = history_training.history["cost"])

In [99]:
if evaluator.accept_cost(cost) is True:

    killed_db, killed_cost, murders = killer.remove_irrelevant_gates(cost,simplified_db)
    simplified_db, ns =  simplifier.reduce_circuit(mutated_db)
    simplified_cost = minimizer.give_cost(simplified_db)
    evaluator.add_step(simplified_db, simplified_cost, relevant=False, operation="simplification", history = ns)

    minimized_db, [cost, resolver, history_training] = minimizer.variational(simplified_db)
    evaluator.add_step(simplified_db, simplified_cost, relevant=True, operation="variational", history = history_training.history["cost"])
    
    circuit_db = minimized_db.copy()